# 手把手打一场NLP赛事

## 0. 版本比较 V2
相较于Ver 1 baseline版本，这个版本中我将按照学习手册进行一些额外的尝试。

自然语言处理（Natural Language Processing，NLP）是指计算机处理和理解人类语言的技术。NLP涵盖了从文本的基本语法和词汇处理到更高级的任务，如机器翻译、情感分析、问答系统等。NLP利用计算机算法和模型，对文本进行分词、词性标注、句法分析等处理，以便将人类语言转化为计算机可以理解和处理的形式

## 1. 赛题信息
医学领域的文献库中蕴含了丰富的疾病诊断和治疗信息，如何高效地从海量文献中提取关键信息，进行疾病诊断和治疗推荐，对于临床医生和研究人员具有重要意义。  


基于论文摘要的文本分类与关键词抽取挑战赛  
https://challenge.xfyun.cn/topic/info?type=abstract-of-the-paper&ch=ymfk4uU

![](https://ai-studio-static-online.cdn.bcebos.com/bc8c545638eb4200a68836ed741b6fe7d75108e9009d443b8de5b33fb8e0fa55)


## 2.准备步骤

### 2.1 赛事报名
赛事地址：https://challenge.xfyun.cn/topic/info?type=abstract-of-the-paper&ch=ZuoaKcY
1. 点击报名参赛，登录讯飞开放平台。

### 2.2 数据下载
数据已提前下载在数据集目录下，您可以自行查看其中的train与test文件

### 2.3 环境配置参考资料
python环境的搭建请参考：
- [Mac设备：Mac上安装Anaconda最全教程](https://zhuanlan.zhihu.com/p/350828057)
- [Windows设备：Anaconda超详细安装教程(Windows环境下)_菜鸟1号!!的博客-CSDN博客_windows安装anaconda](https://blog.csdn.net/fan18317517352/article/details/123035625)
Jupyter 编辑器的使用请参考：
- [Jupyter Notebook最全使用教程，看这篇就够了！](https://www.jianshu.com/p/6cc047bc94e5)

## 3. 赛题解析
实践任务
本任务分为两个子任务：
1. 从论文标题、摘要作者等信息，判断该论文是否属于医学领域的文献。
2. 从论文标题、摘要作者等信息，提取出该论文关键词。

第一个任务看作是一个文本二分类任务。机器需要根据对论文摘要等信息的理解，将论文划分为医学领域的文献和非医学领域的文献两个类别之一。第二个任务看作是一个文本关键词识别任务。机器需要从给定的论文中识别和提取出与论文内容相关的关键词。  

**本次学习中我们仅学习第一个任务**  

数据集解析
训练集与测试集数据为CSV格式文件，各字段分别是标题、作者和摘要。Keywords为任务2的标签，label为任务1的标签。训练集和测试集都可以通过pandas读取。

![](https://ai-studio-static-online.cdn.bcebos.com/8c88537bce9d46049151389396c4c5b828556ba332d34ed3a24555e4e28e7191)


## 4.实践思路&baseline
### 实践思路
本赛题任务主要如下：
1. 从论文标题、摘要作者等信息，判断该论文是否属于医学领域的文献。


#### 任务一：文本二分类
第一个任务看作是一个文本二分类任务。机器需要根据对论文摘要等信息的理解，将论文划分为医学领域的文献和非医学领域的文献两个类别之一。

针对文本分类任务，可以提供两种实践思路，一种是使用传统的特征提取方法（如TF-IDF/BOW）结合机器学习模型，另一种是使用预训练的BERT模型进行建模。使用特征提取 + 机器学习的思路步骤如下：
1. 数据预处理：首先，对文本数据进行预处理，包括文本清洗（如去除特殊字符、标点符号）、分词等操作。可以使用常见的NLP工具包（如NLTK或spaCy）来辅助进行预处理。
2. 特征提取：使用TF-IDF（词频-逆文档频率）或BOW（词袋模型）方法将文本转换为向量表示。TF-IDF可以计算文本中词语的重要性，而BOW则简单地统计每个词语在文本中的出现次数。可以使用scikit-learn库的TfidfVectorizer或CountVectorizer来实现特征提取。
3. 构建训练集和测试集：将预处理后的文本数据分割为训练集和测试集，确保数据集的样本分布均匀。
4. 选择机器学习模型：根据实际情况选择适合的机器学习模型，如朴素贝叶斯、支持向量机（SVM）、随机森林等。这些模型在文本分类任务中表现良好。可以使用scikit-learn库中相应的分类器进行模型训练和评估。
5. 模型训练和评估：使用训练集对选定的机器学习模型进行训练，然后使用测试集进行评估。评估指标可以选择准确率、精确率、召回率、F1值等。
6. 调参优化：如果模型效果不理想，可以尝试调整特征提取的参数（如词频阈值、词袋大小等）或机器学习模型的参数，以获得更好的性能。  


Baseline中我们选择使用BOW将文本转换为向量表示，选择逻辑回归模型来完成训练和评估  
代码演示如下：

## 模块，数据导入与预处理

In [2]:
# 获取前置依赖
!pip install nltk -i https://pypi.org/simple


In [3]:
# 导入pandas用于读取表格数据
import pandas as pd

# 导入BOW（词袋模型），可以选择将CountVectorizer替换为TfidfVectorizer（TF-IDF（词频-逆文档频率）），注意上下文要同时修改，亲测后者效果更佳
from sklearn.feature_extraction.text import TfidfVectorizer
# 导入LogisticRegression回归模型
from sklearn.linear_model import LogisticRegression

# 过滤警告消息
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
## Scikit-learn defines its own set of exceptions that are used to handle specific situations and errors 
## that might occur during the usage of its functions and classes. These exceptions help to 
## provide more informative error messages and to handle exceptional cases gracefully. 
## e.g. "Not Fitted Error","Value Error", "ConvergenceWarning"
## using simplefilter we ignore the non-convergence warning from the model when training.
simplefilter("ignore", category=ConvergenceWarning)

In [20]:
# 读取训练集train和测试集test两个pdDataframe
# 将title和abstract缺失的部分用.fillna('')补全
train = pd.read_csv('/home/aistudio/data/data231041/train.csv')
train['title'] = train['title'].fillna('')
train['abstract'] = train['abstract'].fillna('')
test = pd.read_csv('/home/aistudio/data/data231041/testB.csv')
test['title'] = test['title'].fillna('')
test['abstract'] = test['abstract'].fillna('')

## 训练集包含 6000 文章 (uuid, title, author, abstract, keywords, yes/no); 
## 测试集包含 2000 篇文章 (uuid, title, author, abstract)
# train.shape, test.shape
# train.head()

## 提取文本特征，生成训练集与测试集

In [21]:
## 在训练集train(pdDataFrame)中增添一列series，叫作text，包含文件标题作者摘要和关键词
## 在测试集train(pdDataFrame)中增添一列series，叫作text，包含文件标题作者摘要
## pandas series可以直接求和，批量操作
train['text'] = train['title'].fillna('') + ' ' +  train['author'].fillna('') + ' ' + train['abstract'].fillna('')+ ' ' + train['Keywords'].fillna('')
test['text'] = test['title'].fillna('') + ' ' +  test['author'].fillna('') + ' ' + test['abstract'].fillna('')

train.head()
# test.head()

,uuid,title,author,abstract,Keywords,label,text
0,0,Accessible Visual Artworks for Blind and Visua...,"Quero, Luis Cavazos; Bartolome, Jorge Iranzo; ...",Despite the use of tactile graphics and audio ...,accessibility technology; multimodal interacti...,0,Accessible Visual Artworks for Blind and Visua...
1,1,Seizure Detection and Prediction by Parallel M...,"Li, Chenqi; Lammie, Corey; Dong, Xuening; Amir...","During the past two decades, epileptic seizure...",CNN; Seizure Detection; Seizure Prediction; EE...,1,Seizure Detection and Prediction by Parallel M...
2,2,Fast ScanNet: Fast and Dense Analysis of Multi...,"Lin, Huangjing; Chen, Hao; Graham, Simon; Dou,...",Lymph node metastasis is one of the most impor...,Histopathology image analysis; computational p...,1,Fast ScanNet: Fast and Dense Analysis of Multi...
3,3,Long-Term Effectiveness of Antiretroviral Ther...,"Huang, Peng; Tan, Jingguang; Ma, Wenzhe; Zheng...",In order to assess the effectiveness of the Ch...,HIV; ART; mortality; observational cohort stud...,0,Long-Term Effectiveness of Antiretroviral Ther...
4,4,Real-Time Facial Affective Computing on Mobile...,"Guo, Yuanyuan; Xia, Yifan; Wang, Jing; Yu, Hui...",Convolutional Neural Networks (CNNs) have beco...,facial affective computing; convolutional neur...,0,Real-Time Facial Affective Computing on Mobile...


与Version 1不相同，在计算Tfidf值之前，我们想要先排除一些过于常见的词，称之为停用词。
以下加载停用词,存储在list "stops"中

In [15]:
stops = open(r'stop.txt',encoding='utf-8').readlines()
stops = [i.strip('\n') for i in stops]
print(stops)

["'d", "'ll", "'m", "'re", "'s", "'t", "'ve", 'ZT', 'ZZ', 'a', "a's", 'able', 'about', 'above', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually', 'added', 'adj', 'adopted', 'affected', 'affecting', 'affects', 'after', 'afterwards', 'again', 'against', 'ah', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'announce', 'another', 'any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'apparently', 'appear', 'appreciate', 'appropriate', 'approximately', 'are', 'area', 'areas', 'aren', "aren't", 'arent', 'arise', 'around', 'as', 'aside', 'ask', 'asked', 'asking', 'asks', 'associated', 'at', 'auth', 'available', 'away', 'awfully', 'b', 'back', 'backed', 'backing', 'backs', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'began', 'begin', 'beginning', 'beginnings', 'begins', 'behi

## 计算(word,text) 的TfIdf值

#### 增添一些关于TfidfVectorizer()的使用方法说明：

vector = TfidVectorizer()一个空的实例

vector.fit( ... )
输入是一个Pandas series或者是list，内容为一个系列的纯text。
找出所有在输入系列里存在的所有单词，注意.fit只找出并存储那些单词并给他们标号
输出是一个被fit过的TfidVectorizer实例

vector.transform( ... )
输入是一个Pandas series或者是list，内容为一个系列的纯text。
此处vector需要是一个被fit过的TfidfVectorizer实例，然后它会计算Tfidf值
输出是一个scipy sparse matrix

vector.fit_transform( ... )
将以上两步并了起来。
针对我们的目标，两部需要分开做，具体见代码批注

### 以下我们引入交叉验证集

不同于V1, 我们将原有的训练集train（有是否为医学文献的label）分为70% 30% 的新训练集train2和验证集eval2；

测试集（无label）保持不动。
我们使用新训练集来训练模型并用验证集来评分！

In [59]:
from sklearn.model_selection import train_test_split
train2, eval2 = train_test_split(train, test_size = 0.3)
print("{} is the shape of original table of papers; \n{} and {} are the shapes of training and evaluating sets"
    .format(train.shape,train2.shape,eval2.shape))


(6000, 7) is the shape of original table of papers; 
(4200, 7) and (1800, 7) are the shapes of training and evaluating sets


随后我们开始提取新训练集train2里的所有词汇，提取时，忽略stops中所提到的所有停用词。

将这些词汇fit进一个叫做vector的TfidfVectorizer


In [60]:
# 定义vector为TfidfVectorizer的实例
## 注意此处和Ver 1的不同：vectorizer在被fit的时候只存储不属于stops（停用词）的词汇
vector = TfidfVectorizer(stop_words = stops)
# 取出新训练集train2中'text'为heading的pandas series来fit vector，从而
# 让vector存储series中所提到的所有词汇。vector于是变成fill过的TfidfVectorizer实例
vector = vector.fit(train2['text'])

# 用vector这个fill过的实例中vocabulary的编号来transform三个列表的texts,取得tfidf score存储在分别的稀疏矩阵中
# train2_matrix, eval2_matrix, test_matrix都统一用fit过的vector来作transform
# 这是为了保证同一词汇在两个稀疏矩阵中对应的词汇标号一致(如protein在两个稀疏矩阵里的标号都是67423)
train2_matrix = vector.transform(train2['text'])
eval2_matrix = vector.transform(eval2['text'])
test_matrix = vector.transform(test['text'])

train_matrix和eval_matrix稀疏矩阵存储完毕。

注意到两者列数相同，数字相同是因为在获取这两个稀疏矩阵时统一使用的都是fit好的vector(vectorizer)，vector中包含train['text']中所有vocabulary及对应的列号码。

（但相比于Ver 1中训练集,新训练集更小，总统计到的词更少所以列数少了）

In [62]:
print(train2_matrix.shape)
print(eval2_matrix.shape)
print(test_matrix.shape)

(4200, 54320)
(1800, 54320)
(2000, 54320)


## 引入逻辑回归模型进行训练


In [49]:
from sklearn.metrics import f1_score

In [63]:
model = LogisticRegression()

## 开始训练，这里可以考虑修改默认的batch_size与epoch来取得更好的效果
# Recall：train2['label']是长度为6000*训练ratio 的 1/0，代表是否是医学文献
#         train2_matrix  是一个(6000*训练ratio,....)的稀疏矩阵，存储着每个text中每个单词的Tfidf分
model.fit(train2_matrix, train2['label'])

# 利用模型对测试集label标签进行预测，将预测结果存在eval2的pred_label中，和它的label比较求Fscore
# Recall: eval2['label’]是长度为6000*（1-训练ratio）的 1/0 list
#         eval2_matrix   是一个(6000*(1-训练ratio),....)的稀疏矩阵，。。。
#         eval2['pred_label'] 是预测的6000*（1-训练ratio）的1/0 list
eval2['pred_label'] = model.predict(eval2_matrix)


## 以下会输出F1 score
print(f1_score(eval2['pred_label'],eval2['label']))

0.9184652278177459


In [66]:
eval2

,uuid,title,author,abstract,Keywords,label,text,pred_label
4287,4287,Hybrid SDN evolution: A comprehensive survey o...,"Khorsandroo, Sajad; Gallego Sanchez, Adrian; T...",Software-Defined Networking (SDN) is an evolut...,Software-Defined Networking (SDN); Hybrid SDN;...,0,Hybrid SDN evolution: A comprehensive survey o...,0
3173,3173,Crowd Counting in Low-Resolution Crowded Scene...,"Saqib, Muhammad; Khan, Sultan Daud; Sharma, Na...",Crowd counting and density estimation is an im...,Deep convolutional neural networks; crowd coun...,0,Crowd Counting in Low-Resolution Crowded Scene...,0
3271,3271,Scaling and optimization of MOS optical modula...,"Passaro, Vittorio M. N.; Dell'Olio, Francesco","In this paper, a very accurate model of optica...",integrated optics; MOS capacitors; optical mod...,0,Scaling and optimization of MOS optical modula...,0
823,823,Innovative application research on the combina...,"Feng, Wei; Li, Xiuhua",Under the background of the continuous impact ...,Art design; engineering practice education; in...,0,Innovative application research on the combina...,0
3020,3020,Activation of Angiotensin-converting Enzyme 2 ...,"Priya Tiwari,Virendra Tiwari,Shivangi Gupta,Sh...",Neuroinflammation is associated with activatio...,Angiotensin-Converting Enzyme 2; Glial activat...,1,Activation of Angiotensin-converting Enzyme 2 ...,1
...,...,...,...,...,...,...,...,...
72,72,Severe hypertriglyceridemia in an infant on ch...,"Rasha Aly,Ratna Acharya,Kiran K Upadhyay,Rasha...",Severe hyperlipidemia is a risk factor for car...,hemodialysis; hypertriglyceridemia; infant.,1,Severe hypertriglyceridemia in an infant on ch...,1
5798,5798,Detectability-Based JPEG Steganography Modelin...,"Giboulot, Quentin; Cogranne, Remi; Bas, Patrick",The current art of steganography shows that sc...,Pipelines; Transform coding; Estimation; Discr...,0,Detectability-Based JPEG Steganography Modelin...,0
5654,5654,"The Role of Community Centre-based Arts, Leisu...","Jones, Mat; Kimberlee, Richard; Deave, Toity; ...",Developed countries are experiencing high leve...,community development; arts and health; health...,0,"The Role of Community Centre-based Arts, Leisu...",1
3141,3141,"Pseudomonas oryzagri sp. nov., isolated from a...","Md Amdadul Huq,Sun-Young Lee,Juncai Ma,M Mizan...","A Gram-stain-negative, aerobic, rod-shaped and...",Pseudomonas oryzagri; digital DNA–DNA hybridiz...,1,"Pseudomonas oryzagri sp. nov., isolated from a...",1


可以调试参数，或者改变方法，达到最高F1score 的时候，用fit好的模型来预测test set，再提交

In [65]:
# 用模型预测test_matrix稀疏矩阵最后得到的结果，存在test这个dataFrame中
# 然后，test中需要预测的关键词就直接挑选了标题。。。
test['label'] = model.predict(test_matrix)
test['Keywords'] = test['title'].fillna('')
# 将uuid Keywords 和label 导出生成csv
test[['uuid','Keywords','label']].to_csv('submit_task2.csv', index=None)

完整baseline运行后任务一分数在0.67116左右

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 